In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
ls

gdrive/  sample_data/


In [3]:
cd gdrive/MyDrive/projects-bias-bot/src

/content/gdrive/MyDrive/projects-bias-bot/src


In [4]:
import numpy as np
import pandas as pd
import pickle
import re
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pack_padded_sequence

from tqdm import tqdm
import gensim.downloader as api
from sklearn.metrics import classification_report

from Article import Article

In [5]:
from tqdm import tqdm

In [6]:
class LSTM(nn.Module):
	def __init__(self, input_size, emb_dim, output_size, num_layers, embeds=None):
		super().__init__()
		self.emb = nn.Embedding(input_size, emb_dim)
		if embeds is not None:
			self.emb.weight = nn.Parameter(torch.Tensor(embeds))
		
		self.lstm = nn.LSTM(emb_dim, emb_dim, num_layers=num_layers, bidirectional=True, batch_first=True)
		self.linear = nn.Linear(emb_dim*2, output_size)
		
	def forward(self, input_seq):

		embeds = self.emb( input_seq )

		output_seq , (h_last, c_last) = self.lstm( embeds )

		h_direc_1 = h_last[4,:,:]
		h_direc_2 = h_last[5,:,:]
		h_direc_12 = torch.cat( (h_direc_1, h_direc_2), dim=1 )

		return self.linear(h_direc_12)

In [7]:
def load_vocab(data):
	vocab = dict()
	for item in data:
		for word in item.headline + item.text:
		#for word in item.headline:
			if word in vocab:
				vocab[word] += 1
			else:
				vocab[word] = 1
	vocab = dict(sorted(vocab.items(), key=lambda item: -item[1]))
	return vocab

def make_vocab_dict(vocab):
	word_to_index = {"UNK":0,"FOX":1,"CNN":2,"BBC":3,"Liberal":4,"Conservative":5,"Independent":6,"Other":7}
	count = len(word_to_index)+1
	for word in vocab:
		if word not in word_to_index:
			word_to_index[word] = count 
			count += 1
	return word_to_index

def load_bigrams(data):
	bigram_to_index = dict()
	count = 0
	for article in data:
		full_text = article.headline + article.text
		for i, word in enumerate(full_text):
			if i==0:
				continue
			bigram = (full_text[i-1],word)
			if bigram not in bigram_to_index:
				bigram_to_index[bigram]=count
				count+=1
	return bigram_to_index

def make_unigrams(data, word_to_index, party=None):
	processed_data = []
	for article in data:
		datapoint = []
		if party==None and article.party not in ["Liberal", "Conservative"]:
			continue
		elif article.party != party and party!="Combined":
			continue
		elif party=="Combined":
			datapoint = [word_to_index[article.party]]
		datapoint += [word_to_index[article.source]] + [word_to_index[word] if word in word_to_index else word_to_index["UNK"] for word in article.headline] + [word_to_index[word] if word in word_to_index else word_to_index["UNK"] for word in article.text]
		#datapoint += [word_to_index[article.source]] + [word_to_index[word] if word in word_to_index else word_to_index["UNK"] for word in article.headline]
		label = label_to_index[article.label]

		processed_data.append( (datapoint, label) )
	return processed_data

def make_bigrams(data, bigram_to_index, party):
	processed_data = []
	for article in data:
		if article.party != party:
			continue
		# if article.party not in ["Liberal", "Conservative"]:
		# 	continue

		datapoint = []
		full_text = article.headline + article.text
		for i, word in enumerate(full_text):
			if i==0:
				continue
			datapoint.append(bigram_to_index[(full_text[i-1],word)])
		label = label_to_index[article.label]

		processed_data.append( (datapoint, label) )
	return processed_data
	
def split_data(processed_data):
	return processed_data[:math.floor(0.9*len(processed_data))], processed_data[math.floor(0.9*len(processed_data)):]


def process_batch(batch):
	x = torch.zeros((len(batch), max_len), dtype=torch.long)
	y = torch.zeros((len(batch)), dtype=torch.long)
	for idx, (text, label) in enumerate(batch):
		# print(torch.Tensor(text))
		# print(len(text))
		# print(torch.Tensor(text).size())
		# print()
		x[idx,:len(text)] = torch.Tensor(text)
		y[idx] = label
	return x.to(device), y.to(device)

def get_error(scores, labels):
    bs=scores.size(0)
    predicted_labels = scores.argmax(dim=1)
    indicator = (predicted_labels == labels)
    num_matches=indicator.sum()
    
    return 1-num_matches.float()/bs  

def evaluate(model, test_data):
	with torch.no_grad():
		model.eval()
		x_test, y_test = process_batch(test_data)
	
		pred_y_test = model(x_test)
	
		labels = y_test.tolist()
		predictions = [torch.argmax(pred).item() for pred in pred_y_test]

		print(classification_report(labels, predictions, target_names=["Is Biased","Is Not Biased"], zero_division=0))
		print("Error:",get_error(pred_y_test, y_test).item())



In [10]:
device= torch.device("cuda")
print(device)

# Run only for a single political group
# party = "Liberal"
# party = "Conservative"
party = "Combined"

mode = "unigram"

# Load data
print("Loading data...")
with open("../data/processed_articles.p", "rb") as f:
	data = pickle.load(f)

vocab = load_vocab(data)
print(vocab)

vocab_cutoff=8000
trimmed_vocab=dict()
for i, (word, count) in enumerate(vocab.items()):
	if i <= vocab_cutoff:
		trimmed_vocab[word] = count

word_to_index = make_vocab_dict(trimmed_vocab)
index_to_word = {v: k for k, v in word_to_index.items()}

print(word_to_index)
print(index_to_word)

bigram_to_index = load_bigrams(data)
index_to_bigram = {v: k for k, v in bigram_to_index.items()}


label_to_index = {"is-biased":0, "is-not-biased":1}
max_len = max([len(article.headline + article.text) for article in data]) + 3

print("Creating train data set...")

if mode=="unigram":
	unigrams = make_unigrams(data, word_to_index, party)
	train_data, test_data = split_data(unigrams)
	input_size = len(word_to_index)+1
elif mode=="bigram":
	bigrams = make_bigrams(data, bigram_to_index, party)
	train_data, test_data = split_data(bigrams)
	input_size = len(bigram_to_index)+1



# Hyper parameters
output_size = 2 
num_layers = 3
batch_size = 50
learning_rate = 0.0001
epochs = 20

#Load pre-trained word embeddings, if using them.
# embeds = api.load('glove-twitter-25').vectors
# emb_dim = embeds.shape[1]

embeds = None
emb_dim = 200

# train_data = train_data[0:400]

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=False, collate_fn=process_batch)

print(train_data[0])

num_biased = len([item for item in train_data if item[1]==0])
num_unbiased = len([item for item in train_data if item[1]==1])

print("num biased: ", num_biased)
print("num unbiased: ", num_unbiased)

weights = torch.tensor([num_biased/num_biased,num_biased/num_unbiased]).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)

print("weight: ", weights)

# Build model
model = LSTM(input_size, emb_dim, output_size, num_layers, embeds).to(device)
# criterion = nn.CrossEntropyLoss()

cuda
Loading data...
{'say': 22714, 'not': 14382, 'year': 7331, 'police': 6929, 'court': 6700, 'president': 4719, 'people': 4687, 'accord': 4621, 'time': 4487, 'law': 4375, 'new': 4340, 'tell': 4339, 'school': 4296, 'report': 4210, 'state': 4160, 'news': 4078, 'cnn': 4022, 'ukraine': 3946, 'russian': 3707, 'include': 3636, 'come': 3604, 'fox': 3596, 'russia': 3455, 'justice': 3406, '.': 3355, 'day': 3333, 'official': 3312, 'high': 3205, 'case': 3173, 'biden': 3147, 'statement': 3141, 'charge': 3136, 'trump': 3131, 'department': 3038, 'city': 2992, 'mr': 2968, 'man': 2962, 'officer': 2941, 'go': 2916, 'no': 2905, 'week': 2870, 'woman': 2869, 'house': 2862, 'child': 2833, 'attorney': 2829, 'tuesday': 2814, '-': 2778, 'country': 2760, 'judge': 2754, 'work': 2741, 'war': 2683, 'find': 2610, 'shooting': 2596, 'student': 2483, 'family': 2478, 'take': 2473, 'old': 2470, 'like': 2414, 'call': 2404, 'white': 2390, 'federal': 2344, 'monday': 2320, 'county': 2281, 'texas': 2276, 'jackson': 2185, 

In [ ]:
print(len(data))

4075


In [ ]:
print(len(word_to_index))

8009


In [ ]:
print(len(vocab))

9958


In [ ]:
print(len(train_data))
print(len(test_data))

1519
169


In [ ]:
print(max_len)

1326


In [ ]:
print(len([item for item in train_data if item[1]==1]))
print(len([item for item in train_data if item[1]==0]))

587
932


In [ ]:
for i in range(10):
  item1 = train_data[i]
  print([index_to_word[index] for index in item1[0]])

['FOX', 'nyc', 'career', 'criminal', 'slash', 'man', 'free', 'prior', 'attack', 'fox', 'news', 'flash', 'headline', 'check', 'click', 'foxnews.com', 'violent', 'career', 'criminal', 'allegedly', 'slash', 'stranger', 'face', 'subway', 'sunday', 'release', 'bail', 'similar', 'attack', 'woman', 'brooklyn', 'fox', 'news', 'digital', 'learn', 'brendan', 'dowling', 'allegedly', 'walk', 'year', 'old', 'man', 'sit', 'j', 'train', 'slice', 'open', 'face', 'a.m.', 'march', 'subway', 'car', 'near', 'low', 'east', 'station', 'dowle', 'sixth', 'arrest', 'release', 'bail', 'october', 'plead', 'guilty', 'attempt', 'assault', 'slash', 'woman', 'head', 'december', 'downtown', 'brooklyn', 'accord', 'police', 'prosecutor', 'not', 'immediately', 'clear', 'spring', 'jail', 'brooklyn', 'case', 'face', 'mandatory', 'minimum', ' ', ' ', 'year', 'prison', 'sentence', 'court', 'record', 'dowling', 'arraignment', 'monday', 'manhattan', 'criminal', 'court', 'order', 'hold', 'cash', 'bond', 'charge', 'degree', 'at

In [ ]:
index_to_word[0]

'UNK'

In [ ]:
train_data_subset = train_data[0:10]
evaluate(model, train_data_subset)

               precision    recall  f1-score   support

    Is Biased       0.00      0.00      0.00         5
Is Not Biased       0.50      1.00      0.67         5

     accuracy                           0.50        10
    macro avg       0.25      0.50      0.33        10
 weighted avg       0.25      0.50      0.33        10

Error: 0.5


In [ ]:
# Train loop
for epoch in range(epochs):

	print(f"\n\nEpoch {epoch}")

	if epoch >= 5:
		learning_rate = learning_rate/1.5
	optimizer = optim.Adam(model.parameters(), lr=learning_rate)

	model.train()

	running_error = 0
	count = 0

	for x,y in tqdm(train_dataloader):
   
		if x.size()[0] != batch_size:
			continue 

		# x = x.transpose(0,1)

		scores = model(x)
		scores = scores.view(-1,2)

		loss = criterion(scores, y)
		loss.backward()
		optimizer.step()
		optimizer.zero_grad()

		error = get_error(scores, y)
		running_error += error.item()
		count += 1


	print("\nEvaluate on test:")
	evaluate(model, test_data)
	# print("\nEvaluate on train:")
	# evaluate(model, train_data[0:400])
 

	print("\nRunning Error:", running_error/count)

# Evaluate
torch.save(model.state_dict(),f"model_{party}")






Epoch 0


100%|██████████| 74/74 [02:28<00:00,  2.01s/it]



Evaluate on test:
               precision    recall  f1-score   support

    Is Biased       0.70      0.61      0.65       272
Is Not Biased       0.38      0.47      0.42       136

     accuracy                           0.57       408
    macro avg       0.54      0.54      0.54       408
 weighted avg       0.59      0.57      0.58       408

Error: 0.43382352590560913

Running Error: 0.48712330240092866


Epoch 1


100%|██████████| 74/74 [02:27<00:00,  2.00s/it]



Evaluate on test:
               precision    recall  f1-score   support

    Is Biased       0.76      0.46      0.57       272
Is Not Biased       0.40      0.71      0.51       136

     accuracy                           0.54       408
    macro avg       0.58      0.58      0.54       408
 weighted avg       0.64      0.54      0.55       408

Error: 0.4583333134651184

Running Error: 0.4684931619526589


Epoch 2


...


Evaluate on test:
               precision    recall  f1-score   support

    Is Biased       0.70      0.73      0.71       272
Is Not Biased       0.41      0.38      0.39       136

     accuracy                           0.61       408
    macro avg       0.55      0.55      0.55       408
 weighted avg       0.60      0.61      0.61       408

Error: 0.3897058367729187

Running Error: 0.35945206635618865


Epoch 18


100%|██████████| 74/74 [02:28<00:00,  2.01s/it]



Evaluate on test:
               precision    recall  f1-score   support

    Is Biased       0.70      0.73      0.71       272
Is Not Biased       0.41      0.38      0.39       136

     accuracy                           0.61       408
    macro avg       0.55      0.55      0.55       408
 weighted avg       0.60      0.61      0.61       408

Error: 0.3897058367729187

Running Error: 0.35945206635618865


Epoch 19


100%|██████████| 74/74 [02:28<00:00,  2.01s/it]



Evaluate on test:
               precision    recall  f1-score   support

    Is Biased       0.70      0.73      0.71       272
Is Not Biased       0.41      0.38      0.39       136

     accuracy                           0.61       408
    macro avg       0.55      0.55      0.55       408
 weighted avg       0.60      0.61      0.61       408

Error: 0.3897058367729187

Running Error: 0.35945206635618865


In [ ]:
# model = LSTM(input_size, emb_dim, output_size, num_layers, embeds).to(device)
# model.load_state_dict(torch.load("model_Combined"), strict=False)

In [ ]:
with torch.no_grad():
  model.eval()
  x_test, y_test = process_batch(test_data)
  pred_y_test = model(x_test)
  predictions_with_text = [torch.argmax(pred).item() for pred in pred_y_test]
  labels = y_test.tolist()

In [ ]:
model = LSTM(input_size, emb_dim, output_size, num_layers, embeds).to(device)
model.load_state_dict(torch.load("model_Combined_headlines"), strict=False)

In [ ]:
with torch.no_grad():
  model.eval()
  x_test, y_test = process_batch(test_data)
  pred_y_test = model(x_test)
  predictions_with_headlines = [torch.argmax(pred).item() for pred in pred_y_test]

               precision    recall  f1-score   support

    Is Biased       0.58      0.70      0.63        60
Is Not Biased       0.33      0.23      0.27        40

     accuracy                           0.51       100
    macro avg       0.45      0.46      0.45       100
 weighted avg       0.48      0.51      0.49       100

Error: 0.49000000953674316


In [ ]:
same_result = 0
diff_result = 0
for i, pred_text in enumerate(predictions_with_text):
  pred_headline = predictions_with_headlines[i]

  if pred_text == pred_headline:
    same_result += 1
  else:
    diff_result += 1

In [ ]:
print(same_result)
print(diff_result)

In [ ]:
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False, collate_fn=process_batch)

In [ ]:
def assign_label(party, source, text):
  sentence = [party] + [source] + text.split()
  tokenized = [word_to_index[word] if word in word_to_index else word_to_index["UNK"] for word in sentence]
  label = 0 
  data = [(tokenized,label)]
  x,y = process_batch(data)
  x = x.view(-1,1)
  scores = model(x)
  predicted_label = torch.argmax(scores).item() 

  if predicted_label == 1:
    return "Un-biased"
  else:
    return "Biased"

In [ ]:
assign_label("Independent","BBC","test the model test test")

NameError: ignored

In [ ]:
assign_label("Conservative","CNN","all cops are bad")

'Biased'